In [ ]:
#|default_exp embeddings

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#| export

from clip_plot.utils import timestamp, clean_filename

from pathlib import Path

import torch
import timm
from accelerate import Accelerator

accelerator = Accelerator()

from tqdm.auto import tqdm
import numpy as np

# Create and/or load embeddings

In [ ]:
#| export

def timm_embed_model(model_name: str):
    '''
    Load model and image transform to create embeddings
    Reference: https://huggingface.co/docs/timm/main/en/feature_extraction#pooled

    input:          model name as found in timm documentation
    return tuple:   pre-trained embedding model,
                    transform function to prep images for inference
    '''

    m = timm.create_model(model_name, pretrained=True, num_classes=0)
    m.eval()

    # Reference on transform: https://huggingface.co/docs/timm/main/en/feature_extraction#pooled
    t = timm.data.create_transform(
        **timm.data.resolve_data_config(m.pretrained_cfg)
    )
    print(f"CUDA is available? {torch.cuda.is_available()}")
    return m, t

def timm_transform_embed(img, model, transform, device, dtype) -> np.ndarray:
    '''
    apply transform to image and run inference on it to generate an embedding

    input:      img: Pillow image or similar
                model: Torch model
                transform: Torch image transformation pipeline to match how model was trained
    returns: embedding vector as 1D numpy array
    '''
    img = transform(img).to(device, dtype).unsqueeze(0)
    emb = model(img)
    return emb.detach().cpu().numpy().squeeze()

def get_timm_embeds(imageEngine, model_name: str, **kwargs):
    '''
    Create embedding vectors for input images using a pre-trained model from timm
    '''
    # for now, the output directory is still called "inception" though it is generic
    vector_dir = Path(kwargs["out_dir"]) / "image-vectors" / "inception"
    vector_dir.mkdir(exist_ok=True, parents=True)

    torch.manual_seed(kwargs["seed"])

    print(timestamp(), f"Creating embeddings using {model_name}")
    embeds = []

    model, transform = timm_embed_model(model_name)

    # make some efficiency tweaks to model
    device = accelerator.device
    torch_dtype = torch.float16
    model = accelerator.prepare(model)
    model = model.to(device, torch_dtype)

    for img in tqdm(imageEngine, total=imageEngine.count):
        embed_path = vector_dir / (clean_filename(img.path) + ".npy")
        if embed_path.exists() and kwargs["use_cache"]:
            emb = np.load(embed_path)
        else:
            # create embedding for one image
            emb = timm_transform_embed(img.original, model, transform, device, torch_dtype)
            np.save(embed_path, emb)
        embeds.append(emb)
    return np.array(embeds)

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()